In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import glob

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox

## Select folders

In [ ]:
root = tk.Tk()
root.withdraw()
messagebox.showinfo("Folder Selection", "Press OK and then select the master folder (raw images)")

photos_dir_master = filedialog.askdirectory(title="Select the master folder")
photos_dir_master

In [ ]:
if not photos_dir_master:
    raise ValueError("No input provided")

In [ ]:
# folder containing csv files with results from ImageJ
messagebox.showinfo("Folder Selection", "Press OK and then select the results folder (containing all csv files to be analyzed)")
results_dir = filedialog.askdirectory(title="Select the folder containing resulting csv files from ImageJ")
results_dir

In [ ]:
if not results_dir:
    raise ValueError("No input provided")

# Run cells below

In [ ]:
# folder path you want to store processed results in
output_dir = os.path.join(os.path.dirname(results_dir), "Processed Results_cellarea")
os.makedirs(output_dir)

In [ ]:
images_per_figure = []
figures = [folder for folder in os.listdir(photos_dir_master) if os.path.isdir(os.path.join(photos_dir_master, folder))]

In [ ]:
results = glob.glob(os.path.join(results_dir, "*.csv"))

In [ ]:
cellarea_dct_sq = {}
for i in range(len(figures)):
    csv_filename = os.path.join(results_dir, figures[i] + ".csv")
    cellnums = pd.read_csv(csv_filename, index_col=0)
    image_names_i = glob.glob(os.path.join(photos_dir_master + "/" + figures[i], "**", "*.tif"), recursive=True)
    image_names_i = [j.split("/")[-1] for j in image_names_i]
    cellarea_dct = {j: sum(cellnums.loc[cellnums["Label"] == f"outliers_{j}", "Area"]) for j in image_names_i}
    cellarea_dct_sq.update({figures[i]: cellarea_dct})

In [ ]:
cellarea_dct_sq

In [ ]:
names = [f"{i}-{j}" for i in range(1,4) for j in range(1,4)]
names_br = list(range(1,4))
#image_size = 4080 * 3072

for folder in cellarea_dct_sq.keys():
    cellarea_df = pd.DataFrame.from_dict(cellarea_dct_sq[folder], orient="index").reset_index()
    cellarea_df.columns = ["image_name", "cell_area"]
    cellarea_df["cell_area"] = cellarea_df["cell_area"] #/ image_size
    
    
    df = cellarea_df.copy()
    df.set_index("image_name", inplace=True)
    reform_idx = [j.lower() for j in df.index]
    df.index = reform_idx
    print(df)
    bio_groups = np.unique([j[:-7] for j in reform_idx])
    new_idx = [f"{bio_group}{name}.tif" for bio_group in bio_groups for name in names]
    new_dct = {new_id: df.loc[new_id, "cell_area"] for new_id in new_idx}
    new_vals = np.array(list(new_dct.values())).reshape(-1,3)
    new_df = pd.DataFrame(new_vals, columns=["TR1", "TR2", "TR3"], 
                          index=[f"{bio_group}{k}" for bio_group in bio_groups for k in names_br]).\
                            reset_index()
    new_df["average"] = new_df.loc[:, ["TR1", "TR2", "TR3"]].mean(axis=1)

    with pd.ExcelWriter(f"{output_dir}/{folder}_processed.xlsx") as f:
        cellarea_df.to_excel(f, sheet_name="cell_area")
        new_df.to_excel(f, sheet_name="re-formatted")

        

In [ ]:
new_df